# <center>Unsupervised Learing: Text Clustering</center>

References:
* https://www-users.cs.umn.edu/~kumar/dmbook/ch8.pdf
* http://infolab.stanford.edu/~ullman/mmds/ch7.pdf

## 1. Clustering vs. Classification
* Clustering (Unsupervised): divide a set of objects into clusters (parts of the set) so that objects in the same cluster are similar to each other, and/or objects in different clusters are dissimilar.
  * Representation of the objects
  * Similarity/distance measure
* Classifification (Supervised): group objects into predetermined categories
  * Representation of the objects
  * A training set

<font color=red size = 15>Clustering 我们不知道groudtruth label</font>


## 2. Why clustering
* Understand conceptually meaningful groups of objects that share common characteristics 
* Provides an abstraction from individual data objects to the clusters in which those data objects reside
* Uses of clustering
  * Summarization
  * Compression
  * Efficiently finding nearest neighbors

## 3. Types of Clusterings 
* Different kinds of models (https://www.geeksforgeeks.org/different-types-clustering-algorithm/):
  - <font color=red>Centroid models (partition) =  K-Means - 由中心决定类:</font> 
     - Similarity is derived as the closeness of a data point to the centroid of clusters. 
     - Flat partition, e.g. K-Means
     <img src='centroid.png' width="40%">
  - Connectivity models (Hierarchical algorithms) = bottom up!: 
     - Data points closer in data space exhibit more similarity to each other than the data points lying farther away.      
     -  <font color=red>Hierarchy of clusters</font> , e.g. agglomerative clustering
     <img src='connectivity.png' width="40%">
  - Distribution models: 
     - How probable is it that all data points in the cluster belong to the same distribution, concept, or topic
     - e.g.<font color=red> Latent Semantics Analysis = PCA (在文字处理中, 就是叫LSA)</font>, Latent Dirichlet Allocation (LDA)
     <img src='distribution.png' width="40%">
  - <font color = read> Density models = outlier detection</font>: clusters correspond to areas of varied density of data points in the data space - 一般他们在sparse的地方, 所以可以称之为outlier
     - e.g. DBSCAN
     <img src='density.png' width="40%">
* Exclusive vs. Overlapping
  - Exclusive: each object is assigned to a single cluster, e.g. K-Means
  - Overlapping (non-exclusive): an object can simultaneously belong to more than one cluster, e.g. LDA

## 4. Evaluation of Clustering: What is a good clustering
### 4.1 External Evaluation: 
* External evaluation measures the degree to which predicted clustering labels correspond to actual class labels  
* **Precision** and **Recall**

### 4.2. Internal Evaluation 
<img src='cohension_separation.png' width="60%">
* **Cohension (Intra-cluster similarity)**: how "cohesive" a cluster is, i.e. the average similarity of objects in the same cluster. 
   - e.g. cluster radius: $\max{d(x, μ_A)}$ where $μ_A$ is the arithmetic mean of cluster A and $x$ is a point in A
   - e.g. cluster diameter: $\max{d(x, y)}$ where $x,y$ are two points in cluster A

* **Separation (Inter-cluster dissimilarity)**: how "separate" a cluster from another, i.e. the average similarity of all samples in cluster $A$ to all the samples in cluster $B$.
   - e.g. Separation can be calculated as average distance: $\frac{1}{|A|*|B|}\sum_{x \in A}{\sum_{y \in B}{d(x, y)}}$ 
* Metrics with combined cohension and separation (http://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation)
   - Silhouette Coefficient: $s=\frac{b-a}{\max(a,b)}$, where $a$: the mean distance between a sample and all other points in the same cluster, and $b$: the mean distance between a sample and all other points in the next nearest cluster. $s \in [-1, 1]$.
   - Calinski-Harabaz Index: $s=\frac{b}{a}$ where $a$ is mean within\-cluster separation, and $b$ is the mean between\-cluster separation

## 5. K-Means
### 5.1. Algorithm outline: Cluster objects into K clusters
<img style="float: left;" src='Kmean1.png'  width='20%'/><img  src='Kmean2.png' width='20%'/>
- Algorithm: 
    1. Select K points as initial centroids 
    2. Repeat until centroids do not change:
        1. Form K clusters by assigning each point to its closest centroid by distance.
        2. Recompute the centroid of each cluster as the arithmetic mean of samples within the cluster. 
- A few observations of K-means:
  - Initial centroids have an impact on clustering. Usually, several rounds of clustering with random initial centroids are performed, and the most commonly occurring output centroids are chosen.
  - Centroids and distance measure are crtical in the algorithm
     - **Euclidean distance**: 
       - The best centroid for minimizing the average distance from all samples to the centroid is the mean of points in the cluster (https://www-users.cs.umn.edu/~kumar001/dmbook/ch8.pdf)
       - Curse of dimensionality
       - Sensitive to outliers
     - **Cosine similarity**: 
       * Well-accepted similarity measure for documents
       * It is not guaranteed that the mean of samples in a cluster is the best centroid 
       * For text clustering, the centroid does not stand for an actual document. How to interpret clusters?
     - A modified version of Kmeans is called **K-medoids = 一个点作为中心 real**, where a representative sample is choosen as the center of a cluster, called as a medoid.  **pyclustering 中包含了这个**
- Python packages for Kmean
  * NLTK: can choose Euclidean or Cosine similarity as distance measure
  * Sklearn: only Euclidean distance is supported  
  
  # kmeans 对outlier非常敏感, 需要先处理, 或者normalization, 或者对频数进行可视化, 删除高频词= stop word, 低频词 = outlier
  
  # 在文本处理的distance 需要用cosine similiarity 而不是L2距离
  

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Exercise 5.1.1 Load data and generate TF-IDF
# Load datasets (http://qwone.com/~jason/20Newsgroups/)
# A subset is loaded

import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_excel("C:/Users/Verdi/OneDrive - stevens.edu/Stevens BIA/660/课件/课件6-clustering/twenty_news_data.xlsx")

# Select three labels for now
labels =['comp.graphics', 'soc.religion.christian',\
         'sci.med']
data=df[df["label"].isin(labels)]

# Split dataset into training and test. 
# Assuming we only know ground-truth label 
# for the test set.

train, test = train_test_split(data, test_size=0.2, random_state=0) # 如果没有label, 如何测试我们的model分类的好坏.

# print out the full text of the first sample
print(data["text"][0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



In [3]:
# Exercise 5.1.2
# initialize the TfidfVectorizer 
# set min document frequency to 5

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.corpus import stopwords

# set the min document frequency to 5
# generate tfidf matrix
tfidf_vect = TfidfVectorizer(stop_words="english",\
                             min_df=5) 

dtm= tfidf_vect.fit_transform(train["text"])
print (dtm.shape)

(254, 1257)


In [19]:
# Exercise 5.1.3 Clustering using NLTK KMean
# cosine distance is calculated

from nltk.cluster import KMeansClusterer, \
cosine_distance

# set number of clusters
num_clusters=3

# initialize clustering model
# using cosine distance
# clustering will repeat 20 times
# each with different initial centroids
clusterer = KMeansClusterer(num_clusters, \
                            cosine_distance, \
                            repeats=20)

# samples are assigned to cluster labels 
# starting from 0
clusters = clusterer.cluster(dtm.toarray(),
                             assign_clusters=True)
 # 聚类函数, 只接受dense array, 不接受sparse array?????
    
#print the cluster labels of the first 5 samples  #  每次运行的结果都不一样
print(clusters[0:10])

[1, 1, 2, 0, 1, 0, 0, 0, 1, 2]


In [5]:
# Exercise 5.1.4 Interpret each cluster by centroid

# a centroid is the arithemtic mean 
# of all samples in the cluster
# it may not stand for a real document

# find top words at centroid of each cluster
from sklearn import metrics
import numpy as np

# clusterer.means() contains the centroids
# each row is a cluster, and 
# each column is a feature (word)
centroids=np.array(clusterer.means())

# argsort sort the matrix in ascending order 
# and return locations of features before sorting
# [:,::-1] reverse the order
sorted_centroids = centroids.argsort()[:, ::-1] 

# The mapping between feature (word)
# index and feature (word) can be obtained by
# the vectorizer's function get_feature_names()
voc_lookup= tfidf_vect.get_feature_names() # 

for i in range(num_clusters):
    
    # get words with top 20 tf-idf weight in the centroid
    top_words=[voc_lookup[word_index] \
               for word_index in sorted_centroids[i, :20]]
    print("Cluster %d:\n %s " % (i, "; ".join(top_words)))

Cluster 0:
 edu; nntp; host; posting; graphics; subject; organization; lines; university; thanks; software; com; files; information; washington; file; points; know; video; ca 
Cluster 1:
 edu; pitt; gordon; geb; banks; cs; com; article; msg; writes; science; pain; noring; jim; symptoms; subject; health; cancer; reply; organization 
Cluster 2:
 god; church; jesus; edu; believe; hell; people; christian; catholic; think; faith; bible; com; christians; question; does; rutgers; life; subject; like 


# center的最大N个值所在列, 可以作为该类的topic 组成词

### 5.2. How to evaluate clustering
- External evaluation:
  - Obtain "ground truth": if data is not labeled, manually label a random subset of samples as "ground truth" 
  - Assign each cluster to a "true" class by the **majority vote rule**
  - Calculate precision and recall
  
  
  | Cluster ID      | Ground Truth Class Label   |
  | :------------- |:----------------------------|
  | 0      | comp.graphics|
  | 1      | sci.med  |
  | 2      | soc.religion.christian|
  
- Internal evaluation
  - Silhouette Coefficient
  - Calinski-Harabaz Index
  - ...

In [14]:
# Exercise 5.2.1 Predict labels for new samples

# Question: how to determine 
# the label for a new sample?

# note transform function is used
# not fit_transform
test_dtm = tfidf_vect.transform(test["text"]) # 直接transform 成tf-idf

predicted = [clusterer.classify(v) for v in test_dtm.toarray()]

predicted[0:10]

[0, 1, 0, 1, 1, 1, 1, 1, 2, 0]

In [15]:
# Exercise 5.2.2 External evaluation
# determine cluster labels and calcuate precision and recall

# Create a dataframe with cluster id and 
# ground truth label
confusion_df = pd.DataFrame(list(zip(test["label"].values, predicted)),\
                            columns = ["label", "cluster"])
confusion_df.head()

# generate crosstab between clusters and true labels  = 数据透视表
pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)

,label,cluster
0,comp.graphics,0
1,sci.med,1
2,soc.religion.christian,0
3,sci.med,1
4,sci.med,1


label,comp.graphics,sci.med,soc.religion.christian
cluster,,,
0,21,2,3
1,0,21,0
2,0,0,17


In [8]:
# Exercise 5.2.3 
# Map cluster id to true labels by "majority vote"
cluster_dict={0:'comp.graphics',\
              1:"sci.med",\
              2:'soc.religion.christian'}

# Map true label to cluster id
predicted_target=[cluster_dict[i] \
                  for i in predicted]

print(metrics.classification_report\
      (test["label"], predicted_target))

                        precision    recall  f1-score   support

         comp.graphics       0.81      1.00      0.89        21
               sci.med       1.00      0.91      0.95        23
soc.religion.christian       1.00      0.85      0.92        20

           avg / total       0.94      0.92      0.92        64



### 5.3. Clustering with sklearn package - Euclidean distance
- Compare its performance with NLTK Kmeans result
- Discuss: the difference between performance

In [20]:
# Exercise 5.3.1 Clustering with sklearn package - Euclidean distance
from sklearn.cluster import KMeans

# Kmeans with 20 different centroid seeds
km = KMeans(n_clusters=num_clusters, n_init=20)\
.fit(dtm)  # NLTK 中n_init = interation.  这里
clusters = km.labels_.tolist()


In [21]:
# Exercise 5.3.2 Performance Evaluation

predicted = km.predict(test_dtm)

confusion_df = pd.DataFrame(list(zip(test["label"].values, predicted)),\
                            columns = ["label", "cluster"])
confusion_df.head()

# generate crosstab between clusters and true labels
pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)

,label,cluster
0,comp.graphics,1
1,sci.med,1
2,soc.religion.christian,1
3,sci.med,1
4,sci.med,1


label,comp.graphics,sci.med,soc.religion.christian
cluster,,,
0,0,0,12
1,21,18,8
2,0,5,0


In [11]:
# 5.3.3 Change the mapping accordingly
cluster_dict={1:'comp.graphics', 0:"sci.med",\
              2:'soc.religion.christian'}

# Map true label to cluster id
predicted_target=[cluster_dict[i] \
                  for i in predicted]

print(metrics.classification_report\
      (test["label"], predicted_target))


                        precision    recall  f1-score   support

         comp.graphics       0.43      1.00      0.60        21
               sci.med       0.00      0.00      0.00        23
soc.religion.christian       0.00      0.00      0.00        20

           avg / total       0.14      0.33      0.20        64



- You may notice the significant performance difference caused by:
   - different distance measures: L2 distance vs. Cosine distance
   - high dimensionalties (curse of dimensionality)
- What could be possible ways to solve "curse of dimensionality"?


### 5.4. How to pick *K*, the number of clusters?
- **Try external valuation first!!!**
  - manually assess a subset of documents to create "ground truth"
- In case it is impossible to figure out how many clusters in the data set manually, **theorectically**, *K* may be selected as follows:
  * Select a metric to measure the "goodness" of clusters, e.g. average radius, average diameter, etc.
  * Varying *K* from 2 to N, perform clustering for each *K*
  * Ideally, as *K* increases to some point, the metric should grow slowly (**elbow method**)

<img style="float: left;" src='best_k.png'  width='40%'/><img  src='sample1.png' width='30%'/>
source: http://infolab.stanford.edu/~ullman/mmds/ch7.pdf
- However, if samples do not have clear structures, this method may not work (elbow does not exist!)
<img src="samples2.png" width='30%'>

# 在类别足够多的时候, 在多的分类, 不会影响类的半径. 因为他们是平行的